Simple logistic regression with pytorch, using only one layer

In [25]:
import torch
from torch import *
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import builtins
range = builtins.range


Import the dataset from sklearn, and perform EDA

In [12]:
bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target

In [160]:
# print(X.shape) # data has 30 featues
# print(y) # binary data of 0 and 1
# print(X_train[:10])

In [13]:
n_samples, n_features = X.shape
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

In [14]:
# scaling training and testing data
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.int64))
y_test = torch.from_numpy(y_test.astype(np.int64))

PyTorch workflow:

In [15]:
class LogisticRegression(nn.Module):
    def __init__(self, n_input_features):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(n_input_features, 2)
    
    def forward(self, x):
        logits = self.linear(x)
        return logits

In [31]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'device = {device}')
model = LogisticRegression(n_features).to(device)

device = cuda:0


Loss and Stochastic Gradient Descent

In [17]:
lr = 0.01
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = lr)

In [165]:
print(y_train.shape)
print(y_train.squeeze().long().shape)
print(type(X_train))

torch.Size([455])
torch.Size([455])
<class 'torch.Tensor'>


In [18]:
num_epochs = 100

for epoch in range(num_epochs):
    X_train, y_train = X_train.to(device), y_train.to(device)
    logits = model(X_train)

    loss = loss_fn(logits, y_train.squeeze().long())

    loss.backward()

    optimizer.step()

    #pytorch accumulates gradients by default, we have to reset it 
    optimizer.zero_grad()

    print(f'epoch: {epoch+1}, loss = {loss.item():.4f}')
    

epoch: 1, loss = 0.9396
epoch: 2, loss = 0.8811
epoch: 3, loss = 0.8283
epoch: 4, loss = 0.7808
epoch: 5, loss = 0.7381
epoch: 6, loss = 0.6998
epoch: 7, loss = 0.6654
epoch: 8, loss = 0.6346
epoch: 9, loss = 0.6068
epoch: 10, loss = 0.5817
epoch: 11, loss = 0.5590
epoch: 12, loss = 0.5384
epoch: 13, loss = 0.5196
epoch: 14, loss = 0.5024
epoch: 15, loss = 0.4866
epoch: 16, loss = 0.4721
epoch: 17, loss = 0.4586
epoch: 18, loss = 0.4462
epoch: 19, loss = 0.4346
epoch: 20, loss = 0.4237
epoch: 21, loss = 0.4136
epoch: 22, loss = 0.4041
epoch: 23, loss = 0.3952
epoch: 24, loss = 0.3868
epoch: 25, loss = 0.3788
epoch: 26, loss = 0.3713
epoch: 27, loss = 0.3642
epoch: 28, loss = 0.3574
epoch: 29, loss = 0.3510
epoch: 30, loss = 0.3449
epoch: 31, loss = 0.3390
epoch: 32, loss = 0.3335
epoch: 33, loss = 0.3281
epoch: 34, loss = 0.3230
epoch: 35, loss = 0.3181
epoch: 36, loss = 0.3135
epoch: 37, loss = 0.3090
epoch: 38, loss = 0.3046
epoch: 39, loss = 0.3005
epoch: 40, loss = 0.2965
epoch: 41

Evaluate the model on testing set

In [19]:
model.eval()

with torch.no_grad():
    X_test, y_test = X_test.to(device), y_test.to(device)
    logits = model(X_test)
    y_predicted = torch.argmax(logits, dim = 1)

    corrects = (y_predicted == y_test).sum().item()
    print(f'correct = {corrects}')

    totals = y_test.size(0)
    print(f'totals = {totals}')

    acc = corrects/totals
    print(f'accuracy = {acc * 100:.2f}%')

correct = 105
totals = 114
accuracy = 92.11%


In [34]:
#Your answer here
class model_training():

    def __init__(self, learning_rate: float, model: LogisticRegression, num_epoch: int, training_data, training_features, test_data, test_features):
        self.loss_fn = nn.CrossEntropyLoss()
        self.optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)
        self.X_train = training_data
        self.y_train = training_features

        self.X_test = test_data
        self.y_test = test_features

        self.num_epochs = num_epoch

        self.model = model
    
    def training_func(self):
        for epoch in range(self.num_epochs):
            self.X_train, self.y_train = self.X_train.to(device), self.y_train.to(device) #load the data to device (GPU or CPU)
            # compute logits using the model
            logits = self.model(self.X_train)

            loss = self.loss_fn(logits, self.y_train.squeeze().long())

            # backward pass to compute the gradient
            loss.backward()

            # updates the model parameter based on the gradient
            self.optimizer.step()

            # zero gradients
            self.optimizer.zero_grad()
            
            print(f'epoch: {epoch+1}, loss = {loss.item():.4f}')

    
    def evaluation_func(self):
        # Ensure the model is in evaluation mode
        self.model.eval()

        # Disable gradient calculation
        with torch.no_grad():
            # Load the data to the device (GPU or CPU)
            self.X_test, self.y_test = self.X_test.to(device), self.y_test.to(device)
            # Get the model's predictions
            logits = self.model(self.X_test.type(torch.float32))
            # Compute the predicted class
            y_predicted = torch.argmax(logits, dim=1)

            # Calculate the number of correct predictions
            corrects = (y_predicted == self.y_test).sum().item()
            print(f'correct = {corrects}')

            # Get the total number of samples
            totals = self.y_test.size(0)
            print(f'totals = {totals}')

            # Compute the accuracy
            acc = corrects / totals
            print(f'accuracy = {acc * 100:.2f}%')


In [35]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
log_model = LogisticRegression(n_features).to(device)
trained_model = model_training(0.001, log_model, 30, X_train, y_train, X_test, y_test)

In [36]:
trained_model.training_func()

epoch: 1, loss = 0.6017
epoch: 2, loss = 0.5988
epoch: 3, loss = 0.5959
epoch: 4, loss = 0.5931
epoch: 5, loss = 0.5903
epoch: 6, loss = 0.5876
epoch: 7, loss = 0.5848
epoch: 8, loss = 0.5821
epoch: 9, loss = 0.5795
epoch: 10, loss = 0.5768
epoch: 11, loss = 0.5742
epoch: 12, loss = 0.5716
epoch: 13, loss = 0.5691
epoch: 14, loss = 0.5665
epoch: 15, loss = 0.5640
epoch: 16, loss = 0.5616
epoch: 17, loss = 0.5591
epoch: 18, loss = 0.5567
epoch: 19, loss = 0.5543
epoch: 20, loss = 0.5519
epoch: 21, loss = 0.5496
epoch: 22, loss = 0.5472
epoch: 23, loss = 0.5449
epoch: 24, loss = 0.5426
epoch: 25, loss = 0.5404
epoch: 26, loss = 0.5382
epoch: 27, loss = 0.5359
epoch: 28, loss = 0.5338
epoch: 29, loss = 0.5316
epoch: 30, loss = 0.5294


In [37]:
trained_model.evaluation_func()

correct = 83
totals = 114
accuracy = 72.81%
